In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
path = "gdrive/My Drive/unrestricted_hcp_freesurfer.csv"

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
data_1 =  pd.read_csv(path) 

NameError: name 'path' is not defined

In [0]:
# data_2 =  pd.read_csv("unrestricted_max.csv") 

# 1. Label encoding

In [0]:
encoded_data, mapping_index = pd.factorize(data_1['Release'])

In [0]:
data_1['Release'] = encoded_data

In [0]:
y1 = data_1['Gender']
X1 = data_1.drop(['Gender',], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size = 0.2, stratify = y1, random_state = 0)

## 1.1 XGB

In [63]:
xgb_cls = xgb.XGBClassifier()
xgb_cls.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [64]:
pred = xgb_cls.predict(X_test)
accuracy_score(pred, y_test)

0.9327354260089686

### GridSearchCV

In [0]:
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold,GroupKFold
from sklearn.model_selection import GridSearchCV

In [0]:
kfold = StratifiedKFold(n_splits=5, shuffle=True) # 5-fold CV
grid = { 'max_depth':[3,5,10],
    'n_estimators': [100,200,300],
    'learning_rate':[0.02, 0.1, 0.5]}
gs_xgb = GridSearchCV(xgb.XGBClassifier(), grid, n_jobs=-1 , cv=kfold)

In [68]:
gs_xgb.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
             error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'learning_rate': [0.02, 0.1, 0.5],
                         'max_depth': [3, 5, 10

In [69]:
gs_xgb.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.5, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [70]:
pred = gs_xgb.predict(X_test)
accuracy_score(pred, y_test)

0.9237668161434978

### Cross_val_score

In [0]:
c = cross_val_score(xgb.XGBClassifier(), X1, y1, cv=5)
c, c.mean()

In [0]:
c = cross_val_score(xgb.XGBClassifier(), X1, y1, cv=10)
c, c.mean()

(array([0.89285714, 0.86607143, 0.90178571, 0.87387387, 0.89189189,
        0.89189189, 0.90990991, 0.9009009 , 0.85585586, 0.87387387]),
 0.8858912483912483)

## 1.2 KNN

In [0]:
knn_clf = KNeighborsClassifier()

In [0]:
knn_clf.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [0]:
pred = knn_clf.predict(X_test)

In [0]:
accuracy_score(pred, y_test)

0.8385650224215246

In [0]:
c = cross_val_score(KNeighborsClassifier(n_neighbors=15), X1, y1, cv=5)
c, c.mean()

(array([0.86098655, 0.84753363, 0.82959641, 0.75675676, 0.78378378]),
 0.8157314264937583)

In [0]:
c = cross_val_score(KNeighborsClassifier(n_neighbors=15), X1, y1, cv=10)
c, c.mean()

(array([0.85714286, 0.82142857, 0.86607143, 0.84684685, 0.82882883,
        0.84684685, 0.76576577, 0.73873874, 0.79279279, 0.83783784]),
 0.8202300514800515)

## 1.3 LinearRegression

In [0]:
scaler = StandardScaler()

In [0]:
X_t = scaler.fit_transform(X_train)

In [0]:
# X_t = X_train.apply(pd.to_numeric, errors='coerce')
# X_t.fillna(0, inplace=True)

In [0]:
lr_clf = LogisticRegression(max_iter=10000)

In [0]:
Y_t = y_train.replace({'M': 1, 'F': 0})

In [0]:
lr_clf.fit(X_t, Y_t)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
# X_te = X_test.apply(pd.to_numeric, errors='coerce')
# X_te.fillna(0, inplace=True)

In [0]:
X_te = scaler.fit_transform(X_test)

In [0]:
Y_test = y_test.replace({'M': 1, 'F': 0})

In [0]:
pred = lr_clf.predict(X_te)

In [0]:
accuracy_score(pred, Y_test)

0.9282511210762332

In [0]:
X_scaled = scaler.fit_transform(X1)
y_scaled = y1.replace({'M': 1, 'F': 0})

In [0]:
c = cross_val_score(LogisticRegression(max_iter=10000), X_scaled, y_scaled, cv=5)
c, c.mean()

(array([0.92376682, 0.86995516, 0.9103139 , 0.90990991, 0.90990991]),
 0.9047711388518562)

In [0]:
c = cross_val_score(LogisticRegression(max_iter=10000), X_scaled, y_scaled, cv=10)
c, c.mean()

(array([0.95535714, 0.91964286, 0.85714286, 0.87387387, 0.94594595,
        0.9009009 , 0.92792793, 0.92792793, 0.90990991, 0.92792793]),
 0.9146557271557272)

# 2. One hot encoding

In [0]:
results_1113 ={'xbg':0, "knn":0, 'lr':0}
results_500 ={'xbg':0, "knn":0, 'lr':0}
results_10_1113 ={'xbg':0, "knn":0, 'lr':0}
results_10_500 ={'xbg':0, "knn":0, 'lr':0}

In [0]:
data_1 =  pd.read_csv(path) 

In [0]:
y1 = data_1['Gender']

In [0]:
X1 = data_1.drop(['Gender',], axis = 1)

## 2.1 XGB  

In [0]:
# categorical_attr_names = ['Release']
# data_encoded = pd.get_dummies(X1[categorical_attr_names])

In [0]:
X1 = pd.get_dummies(X1)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size = 0.2, stratify = y1, random_state = 0)

In [0]:
xgb_cls = xgb.XGBClassifier()

In [26]:
xgb_cls.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
pred = xgb_cls.predict(X_test)

In [28]:
accuracy_score(pred, y_test)

0.9327354260089686

In [0]:
loo = LeaveOneOut()

In [0]:
c = cross_val_score(xgb.XGBClassifier(), X1, y1, cv=loo)
results_1113['xgb'] = c.mean(), c.std()

In [0]:
c = cross_val_score(xgb.XGBClassifier(), X1[:500], y1[:500], cv=loo)
results_500['xgb'] = c.mean(), c.std()

In [0]:
c = cross_val_score(xgb.XGBClassifier(), X1, y1, cv=10)
results_10_1113['xgb'] = c.mean(), c.std()

In [0]:
c = cross_val_score(xgb.XGBClassifier(), X1[:500], y1[:500], cv=10)
results_10_500['xgb'] = c.mean(), c.std()

## 2.2 KNN

In [0]:
knn_clf = KNeighborsClassifier(n_neighbors=15)
knn_clf.fit(X_train, y_train)
pred = knn_clf.predict(X_test)
accuracy_score(pred, y_test)

0.8654708520179372

In [0]:
c = cross_val_score(KNeighborsClassifier(n_neighbors=15), X1, y1, cv=5)
c, c.mean()

(array([0.86098655, 0.84753363, 0.82959641, 0.75675676, 0.78378378]),
 0.8157314264937583)

In [0]:
c = cross_val_score(KNeighborsClassifier(n_neighbors=15), X1, y1, cv=10)
c, c.mean()

(array([0.85714286, 0.82142857, 0.86607143, 0.84684685, 0.82882883,
        0.84684685, 0.76576577, 0.73873874, 0.79279279, 0.83783784]),
 0.8202300514800515)

In [0]:
c = cross_val_score(KNeighborsClassifier(n_neighbors=15), X1, y1, cv=loo)
results_1113['knn'] = c.mean(), c.std()

In [0]:
c = cross_val_score(KNeighborsClassifier(n_neighbors=15), X1[:500], y1[:500], cv=loo)
results_500['knn'] = c.mean(), c.std()

In [0]:
c = cross_val_score(KNeighborsClassifier(n_neighbors=15), X1, y1, cv=10)
results_10_1113['knn'] = c.mean(), c.std()

In [0]:
c = cross_val_score(KNeighborsClassifier(n_neighbors=15), X1[:500], y1[:500], cv=10)
results_10_500['knn'] = c.mean(), c.std()

## 2.3 LogisticRegression

In [40]:
scaler = StandardScaler()
X_t = scaler.fit_transform(X_train)
Y_t = y_train.replace({'M': 1, 'F': 0})
lr_clf = LogisticRegression(max_iter=10000)
lr_clf.fit(X_t, Y_t)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
X_te = scaler.fit_transform(X_test)
Y_te = y_test.replace({'M': 1, 'F': 0})

In [42]:
pred = lr_clf.predict(X_te)
accuracy_score(pred, Y_te)

0.9372197309417041

In [0]:
X_scaled = scaler.fit_transform(X1)
y_scaled = y1.replace({'M': 1, 'F': 0})

In [44]:
c = cross_val_score(LogisticRegression(max_iter=10000), X_scaled, y_scaled, cv=5)
c, c.mean()

(array([0.92825112, 0.87892377, 0.91479821, 0.90990991, 0.91891892]),
 0.9101603845998465)

In [45]:
c = cross_val_score(LogisticRegression(max_iter=10000), X_scaled, y_scaled, cv=10)
c, c.mean()

(array([0.95535714, 0.91964286, 0.86607143, 0.87387387, 0.94594595,
        0.9009009 , 0.92792793, 0.92792793, 0.90990991, 0.92792793]),
 0.9155485842985843)

In [0]:
c = cross_val_score(LogisticRegression(max_iter=10000), X_scaled, y_scaled, cv=loo)
results_1113['lr'] = c.mean(), c.std()

In [0]:
c = cross_val_score(LogisticRegression(max_iter=10000), X_scaled[:500], y_scaled[:500], cv=loo)
results_500['lr'] = c.mean(), c.std()

In [0]:
c = cross_val_score(LogisticRegression(max_iter=10000), X_scaled, y_scaled, cv=10)
results_10_1113['lr'] = c.mean(), c.std()

In [0]:
c = cross_val_score(LogisticRegression(max_iter=10000), X_scaled[:500], y_scaled[:500], cv=10)
results_10_500['lr'] = c.mean(), c.std()

# Results (Leave one out cross validation)

## 1113 people

In [0]:
import plotly.graph_objects as go

In [56]:
fig = go.Figure(data=[go.Table(header=dict(values=['score', 'xgb', 'knn', 'linear regression']),
                 cells=dict(values=[['mean','std'],[results_1113['xgb'][0],results_1113['xgb'][1]], [results_1113['knn'][0],results_1113['knn'][1]], [results_1113['lr'][0],results_1113['lr'][1]]]))
                     ])
fig.show()

### 500 people

In [57]:
fig = go.Figure(data=[go.Table(header=dict(values=['score', 'xgb', 'knn', 'linear regression']),
                 cells=dict(values=[['mean','std'],[results_500['xgb'][0],results_500['xgb'][1]], [results_500['knn'][0],results_500['knn'][1]], [results_500['lr'][0],results_500['lr'][1]]]))
                     ])
fig.show()

# Results (10-fold cross validation)

In [48]:
fig = go.Figure(data=[go.Table(header=dict(values=['score', 'xgb', 'knn', 'linear regression']),
                 cells=dict(values=[['mean','std'],[results_10_1113['xgb'][0],results_10_1113['xgb'][1]], [results_10_1113['knn'][0],results_10_1113['knn'][1]], [results_10_1113['lr'][0],results_10_1113['lr'][1]]]))
                     ])
fig.show()

In [49]:
fig = go.Figure(data=[go.Table(header=dict(values=['score', 'xgb', 'knn', 'linear regression']),
                 cells=dict(values=[['mean','std'],[results_10_500['xgb'][0],results_10_500['xgb'][1]], [results_10_500['knn'][0],results_10_500['knn'][1]], [results_10_500['lr'][0],results_10_500['lr'][1]]]))
                     ])
fig.show()